<a href="https://colab.research.google.com/github/YianKim/2022_uncertainty_aware_semisupervise/blob/main/uncertaintylabeling_fer2013%2Bresnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 라이브러리 불러오기


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import PIL
import pickle
import random
from tqdm import tqdm
from collections import Counter

from keras.layers.core import Lambda
from keras import backend as K

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import BatchNormalization
from keras.regularizers import l2
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import LSTM
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import *
from keras.models import load_model

from tensorflow import Tensor
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model

from tqdm import tqdm

## 데이터 불러오기


In [4]:
def load_data(dataset_path):
  
  #classes = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprsie', 'Neutral']

  data = []
  test_data = []
  test_labels = []
  labels =[]

  with open(dataset_path, 'r') as file:
      for line_no, line in enumerate(file.readlines()):
          if 0 < line_no <= 35887:
            curr_class, line, set_type = line.split(',')
            image_data = np.asarray([int(x) for x in line.split()]).reshape(48, 48)
            image_data =image_data.astype(np.uint8)/255.0

            if (set_type.strip() == 'PrivateTest'):
              
              test_data.append(image_data)
              test_labels.append(curr_class)
            else:
              data.append(image_data)
              labels.append(curr_class)
      
      test_data = np.expand_dims(test_data, -1)
      test_labels = to_categorical(test_labels, num_classes = 7)
      data = np.expand_dims(data, -1)   
      labels = to_categorical(labels, num_classes = 7)
    
      return np.array(data), np.array(labels), np.array(test_data), np.array(test_labels)

dataset_path = "/content/drive/MyDrive/파이썬스터디 프로젝트/fer2013.csv/fer2013.csv"
train_data, train_labels, test_data, test_labels = load_data(dataset_path)
#train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size = test_size,random_state = seed)

#평가용
test_labels2 = np.argmax(test_labels, axis=1)

# print("Number of images in Training set:", len(train_data))
# print("Number of images in Test set:", len(test_data))

with open('/content/drive/MyDrive/파이썬스터디 프로젝트/aug_array1.pkl', 'rb') as f:
	aug_array1 = pickle.load(f)
 
train_data_aug = np.concatenate((train_data, aug_array1), axis=0)
train_labels_aug = np.concatenate((train_labels, train_labels), axis=0)

In [5]:
train_data_aug.shape

(64596, 48, 48, 1)

In [6]:
train_labels_aug.shape

(64596, 7)

In [32]:
train_data = train_data_aug[range(10000)]

In [33]:
train_labels = train_labels_aug[range(10000)]

In [34]:
unlab_data = train_data_aug[range(10000,64596)]

In [35]:
unlab_labels = train_labels_aug[range(10000,64596)]

## resnet

In [36]:
def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

def relu_bn(inputs: Tensor) -> Tensor:
    relu = ReLU()(inputs)
    bn = BatchNormalization()(relu)
    return bn

def residual_block(x: Tensor, downsample: bool, filters: int, kernel_size: int = 3) -> Tensor:
    y = Conv2D(kernel_size=kernel_size,
               strides= (1 if not downsample else 2),
               filters=filters,
               padding="same")(x)
    y = relu_bn(y)
    y = Conv2D(kernel_size=kernel_size,
               strides=1,
               filters=filters,
               padding="same")(y)

    if downsample:
        x = Conv2D(kernel_size=1,
                   strides=2,
                   filters=filters,
                   padding="same")(x)
    out = Add()([x, y])
    out = relu_bn(out)
    return out

def create_res_net():
    
    inputs = Input(shape=(48, 48, 1))
    num_filters = 128
    
    t = BatchNormalization()(inputs)
    t = Conv2D(kernel_size=3,
               strides=1,
               filters=num_filters,
               padding="same")(t)
    t = relu_bn(t)
    t = PermaDropout(0.5)(t)
    
    num_blocks_list = [2, 5, 5, 2]
    for i in range(len(num_blocks_list)):
        num_blocks = num_blocks_list[i]
        for j in range(num_blocks):
            t = residual_block(t, downsample=(j==0 and i!=0), filters=num_filters)
        num_filters *= 2
        t = PermaDropout(0.5)(t)
    
    t = AveragePooling2D(6)(t)
    t = Flatten()(t)
    outputs = Dense(7, activation='softmax')(t)
    
    model = Model(inputs, outputs)

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [37]:
model = create_res_net() # or create_plain_net()
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 48, 48, 1)]  0           []                               
                                                                                                  
 batch_normalization_120 (Batch  (None, 48, 48, 1)   4           ['input_5[0][0]']                
 Normalization)                                                                                   
                                                                                                  
 conv2d_128 (Conv2D)            (None, 48, 48, 128)  1280        ['batch_normalization_120[0][0]']
                                                                                                  
 re_lu_116 (ReLU)               (None, 48, 48, 128)  0           ['conv2d_128[0][0]']       

In [ ]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')

model.fit(
    x=train_data,
    y=train_labels,
    epochs=500,
    verbose=1,
    validation_split=0.3,
    batch_size=64,
    callbacks=[lr_reducer, early_stopper]
)

Epoch 1/500
110/110 [==============================] - 159s 1s/step - loss: 2.1876 - accuracy: 0.2089 - val_loss: 7.6638 - val_accuracy: 0.2243 - lr: 0.0010
Epoch 2/500
110/110 [==============================] - 129s 1s/step - loss: 1.8900 - accuracy: 0.2207 - val_loss: 3.0755 - val_accuracy: 0.1640 - lr: 0.0010
Epoch 3/500
 34/110 [========>.....................] - ETA: 1:19 - loss: 1.8782 - accuracy: 0.2463

In [ ]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_fer_resnet.h5')

## 모델 결과

In [ ]:
def model_eval():
  pred_mu = model.predict(test_data)
  for i in range(1, 30):
    pred_mu += model.predict(test_data)
  pred_mu = pred_mu/30
  predicted_test_labels = np.argmax(pred_mu, axis=1)
  return(accuracy_score(np.argmax(test_labels, axis=1), predicted_test_labels))

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_fer_resnet.h5')
model_eval()

# 라벨링

## 라벨링; 1

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_fer_resnet.h5')

In [ ]:
# Vars : 분산들
# Outs : 결과 값들

Vars = []
Outs = []

temp1 = []

for i in tqdm(range(20)):
  temp1.append(model.predict(unlab_data))

for j in tqdm(range(unlab_data.shape[0])):

  temp2 = np.array([0,0,0,0,0,0,0], 'float32')
  
  for i in range(30):
    temp2 += temp1[i][j]
  Outs.append(temp2/30)
  
  outputs=[]

  for i in range(30):
    outputs.append(temp1[i][j][np.argmax(temp2)])
  Vars.append(np.var(outputs))

In [ ]:
Outs = np.array(Outs)

class마다 균등하게 선택

In [ ]:
Counter(np.argmax(unlab_labels, axis=1))

### 불확정성


In [ ]:
# upper bound of variance?

UB25 = []
UB50 = []
UB75 = []


for h in range(7):
  classvars = []
  for i in tqdm(range(54596)):
    if np.argmax(unlab_labels, axis=1).tolist()[i]==h:
      classvars.append(Vars[i])
  UB25.append(np.percentile(classvars, 25))
  UB50.append(np.percentile(classvars, 50))
  UB75.append(np.percentile(classvars, 75))

In [ ]:
# UB25 < UB50 < UB75

In [ ]:
vars25 = []
vars50 = []
vars75 = []
vars100 = []

ind = 0 


for i in Vars:
  if i <= UB25[np.argmax(unlab_labels, axis=1)[ind]]:
    vars25.append(ind)
  elif i <= UB50[np.argmax(unlab_labels, axis=1)[ind]]:
    vars50.append(ind)
  elif i <= UB75[np.argmax(unlab_labels, axis=1)[ind]]:
    vars75.append(ind)
  else:
    vars100.append(ind)
  ind += 1

In [ ]:
k1 = random.sample(range(len(vars25)), len(vars25))
k2 = random.sample(range(len(vars50)), len(vars50))
k3 = random.sample(range(len(vars75)), len(vars75))
k4 = random.sample(range(len(vars100)), len(vars100))

lowvars = k1[0:len(k1)]+k2[0:len(k2)]
highvars = k3[0:len(k3)]+k4[0:len(k4)]

clstvars1 = k1[0:np.int(len(k1)/2)] + k2[0:np.int(len(k2)/2)] + k3[0:np.int(len(k3)/2)] + k4[0:np.int(len(k4)/2)]
clstvars2 = k1[np.int(len(k1)/2):len(k1)] + k2[np.int(len(k2)/2):len(k2)] + k3[np.int(len(k3)/2):len(k3)] + k4[np.int(len(k4)/2):len(k4)]

In [ ]:
# 같은 길이의 랜덤하게 고른 data들에 모델로 label 부여 후 정확도 측정
randomindx = random.sample(range(54596), 54596)
randomindx2 = randomindx[0:27298]
randomindx = randomindx[27298:50000]
accuracy_score(np.argmax(np.array(Outs)[randomindx], axis=1), np.argmax(unlab_labels[randomindx], axis=1))

In [ ]:
# 분산추출
accuracy_score(np.argmax(np.array(Outs)[lowvars], axis=1), np.argmax(unlab_labels[lowvars], axis=1))

In [ ]:
# 층화추출
accuracy_score(np.argmax(np.array(Outs)[clstvars1], axis=1), np.argmax(unlab_labels[clstvars1], axis=1))

In [ ]:
train_data_1 = unlab_data[lowvars]
train_labels_1 = Outs[lowvars]

In [ ]:
shufindx = random.sample(range(train_data_1.shape[0]),train_data_1.shape[0])
train_data_1 = train_data_1[shufindx]
train_labels_1 = train_labels_1[shufindx]

In [ ]:
train_data_2 = unlab_data[clstvars1]
train_labels_2 = Outs[clstvars1]

In [ ]:
shufindx = random.sample(range(train_data_2.shape[0]),train_data_2.shape[0])
train_data_2 = train_data_2[shufindx]
train_labels_2 = train_labels_2[shufindx]

In [ ]:
train_data_3 = unlab_data[randomindx]
train_labels_3 = Outs[randomindx]

In [ ]:
shufindx = random.sample(range(train_data_3.shape[0]),train_data_3.shape[0])
train_data_3 = train_data_3[shufindx]
train_labels_3 = train_labels_3[shufindx]

In [ ]:
unlab_data_1 = unlab_data[highvars]
unlab_labels_1 = unlab_labels[highvars]
unlab_data_2 = unlab_data[clstvars2]
unlab_labels_2 = unlab_labels[clstvars2]
unlab_data_3 = unlab_data[randomindx2]
unlab_labels_3 = unlab_labels[randomindx2]

In [ ]:
Counter(np.argmax(train_labels_1, axis=1))

In [ ]:
Counter(np.argmax(train_labels_2, axis=1))

In [ ]:
Counter(np.argmax(train_labels_3, axis=1))

In [ ]:
Counter(np.argmax(unlab_labels_1, axis=1))

In [ ]:
Counter(np.argmax(unlab_labels_2, axis=1))

In [ ]:
Counter(np.argmax(unlab_labels_3, axis=1))

## 모델링

간단한 모델에서 받은 지식를 복잡한 모델에서 학습 

uncertainty aware data vs random sampling data

#### Resnet

In [ ]:
def relu_bn(inputs: Tensor) -> Tensor:
    relu = ReLU()(inputs)
    bn = BatchNormalization()(relu)
    return bn

def create_simple_net():
    
    inputs = Input(shape=(48, 48, 1))
    num_filters = 64
    
    t = BatchNormalization()(inputs)
    t = Conv2D(kernel_size=3,
               strides=1,
               filters=num_filters,
               padding="same")(t)
    t = relu_bn(t)
    t = Conv2D(kernel_size=3,
               strides=1,
               filters=num_filters,
               padding="same")(t)
    t = relu_bn(t)
    t = Dropout(0.5)(t)
    t = Flatten()(t)
    outputs = Dense(7, activation='softmax')(t)
    
    model = Model(inputs, outputs)

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [ ]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
model = create_simple_net()

model.fit(
    x=train_data_1,
    y=train_labels_1,
    epochs=500,
    verbose=1,
    validation_split=0.3,
    batch_size=64,
    callbacks=[lr_reducer, early_stopper]
)

for i in range(len(model.layers)-1):
    model.layers[i].trainable = False
    
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.00001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')

model.fit(
    x=train_data,
    y=train_labels,
    epochs=500,
    verbose=1,
    validation_split=0.3,
    batch_size=64,
    callbacks=[lr_reducer, early_stopper]
)

model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_fer_resnet_a1.h5')

Epoch 1/500
110/110 [==============================] - 2s 13ms/step - loss: 0.2869 - accuracy: 0.9070 - val_loss: 0.3547 - val_accuracy: 0.8943
Epoch 2/500
110/110 [==============================] - 1s 11ms/step - loss: 0.2841 - accuracy: 0.9080 - val_loss: 0.3535 - val_accuracy: 0.8967
Epoch 3/500
110/110 [==============================] - 1s 11ms/step - loss: 0.2849 - accuracy: 0.9066 - val_loss: 0.3525 - val_accuracy: 0.8970
Epoch 4/500
110/110 [==============================] - 1s 9ms/step - loss: 0.2780 - accuracy: 0.9106 - val_loss: 0.3515 - val_accuracy: 0.8977
Epoch 5/500
110/110 [==============================] - 1s 8ms/step - loss: 0.2739 - accuracy: 0.9110 - val_loss: 0.3507 - val_accuracy: 0.8977
Epoch 6/500
110/110 [==============================] - 1s 10ms/step - loss: 0.2718 - accuracy: 0.9100 - val_loss: 0.3499 - val_accuracy: 0.8983
Epoch 7/500
110/110 [==============================] - 1s 11ms/step - loss: 0.2697 - accuracy: 0.9109 - val_loss: 0.3492 - val_accuracy: 0

110/110 [==============================] - 1s 12ms/step - loss: 0.2154 - accuracy: 0.9257 - val_loss: 0.3248 - val_accuracy: 0.9033
Epoch 58/500
110/110 [==============================] - 1s 9ms/step - loss: 0.2112 - accuracy: 0.9253 - val_loss: 0.3243 - val_accuracy: 0.9033
Epoch 59/500
110/110 [==============================] - 1s 11ms/step - loss: 0.2110 - accuracy: 0.9266 - val_loss: 0.3240 - val_accuracy: 0.9033
Epoch 60/500
110/110 [==============================] - 1s 12ms/step - loss: 0.2091 - accuracy: 0.9279 - val_loss: 0.3237 - val_accuracy: 0.9033
Epoch 61/500
110/110 [==============================] - 1s 12ms/step - loss: 0.2074 - accuracy: 0.9274 - val_loss: 0.3234 - val_accuracy: 0.9030
Epoch 62/500
110/110 [==============================] - 1s 12ms/step - loss: 0.2124 - accuracy: 0.9227 - val_loss: 0.3231 - val_accuracy: 0.9030
Epoch 63/500
110/110 [==============================] - 1s 11ms/step - loss: 0.2082 - accuracy: 0.9260 - val_loss: 0.3227 - val_accuracy: 0.9027

0.8991

In [ ]:
model_eval()

In [ ]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
model = create_simple_net()

model.fit(
    x=train_data_2,
    y=train_labels_2,
    epochs=500,
    verbose=1,
    validation_split=0.3,
    batch_size=64,
    callbacks=[lr_reducer, early_stopper]
)

for i in range(len(model.layers)-1):
    model.layers[i].trainable = False
    
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.00001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')

model.fit(
    x=train_data,
    y=train_labels,
    epochs=500,
    verbose=1,
    validation_split=0.3,
    batch_size=64,
    callbacks=[lr_reducer, early_stopper]
)

model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_fer_resnet_b1.h5')

Epoch 1/500
274/274 [==============================] - 4s 14ms/step - loss: 1.6017 - accuracy: 0.8135 - val_loss: 1.6366 - val_accuracy: 0.5920
Epoch 2/500
274/274 [==============================] - 3s 13ms/step - loss: 1.0112 - accuracy: 0.8792 - val_loss: 0.7154 - val_accuracy: 0.8664
Epoch 3/500
274/274 [==============================] - 3s 12ms/step - loss: 0.7849 - accuracy: 0.8987 - val_loss: 0.7207 - val_accuracy: 0.8985
Epoch 4/500
274/274 [==============================] - 3s 12ms/step - loss: 0.6292 - accuracy: 0.9117 - val_loss: 0.5370 - val_accuracy: 0.9220
Epoch 5/500
274/274 [==============================] - 4s 13ms/step - loss: 0.4803 - accuracy: 0.9243 - val_loss: 0.5016 - val_accuracy: 0.9205
Epoch 6/500
274/274 [==============================] - 4s 13ms/step - loss: 0.3818 - accuracy: 0.9381 - val_loss: 0.3587 - val_accuracy: 0.9393
Epoch 7/500
274/274 [==============================] - 3s 13ms/step - loss: 0.3136 - accuracy: 0.9417 - val_loss: 0.3540 - val_accuracy:

274/274 [==============================] - 3s 12ms/step - loss: 0.1023 - accuracy: 0.9834 - val_loss: 0.1466 - val_accuracy: 0.9649
Epoch 58/500
274/274 [==============================] - 3s 12ms/step - loss: 0.1027 - accuracy: 0.9817 - val_loss: 0.1444 - val_accuracy: 0.9692
Epoch 59/500
274/274 [==============================] - 3s 12ms/step - loss: 0.1022 - accuracy: 0.9826 - val_loss: 0.1456 - val_accuracy: 0.9687
Epoch 60/500
274/274 [==============================] - 3s 11ms/step - loss: 0.1027 - accuracy: 0.9821 - val_loss: 0.1449 - val_accuracy: 0.9699
Epoch 61/500
274/274 [==============================] - 3s 12ms/step - loss: 0.1023 - accuracy: 0.9823 - val_loss: 0.1430 - val_accuracy: 0.9691
Epoch 62/500
274/274 [==============================] - 3s 13ms/step - loss: 0.1017 - accuracy: 0.9813 - val_loss: 0.1428 - val_accuracy: 0.9705
Epoch 63/500
274/274 [==============================] - 3s 13ms/step - loss: 0.1019 - accuracy: 0.9827 - val_loss: 0.1446 - val_accuracy: 0.969

274/274 [==============================] - 3s 13ms/step - loss: 0.0936 - accuracy: 0.9842 - val_loss: 0.1379 - val_accuracy: 0.9731
Epoch 114/500
274/274 [==============================] - 4s 13ms/step - loss: 0.0930 - accuracy: 0.9854 - val_loss: 0.1398 - val_accuracy: 0.9719
Epoch 115/500
274/274 [==============================] - 4s 13ms/step - loss: 0.0929 - accuracy: 0.9864 - val_loss: 0.1382 - val_accuracy: 0.9736
Epoch 116/500
274/274 [==============================] - 3s 13ms/step - loss: 0.0926 - accuracy: 0.9862 - val_loss: 0.1396 - val_accuracy: 0.9728
Epoch 117/500
274/274 [==============================] - 3s 13ms/step - loss: 0.0928 - accuracy: 0.9853 - val_loss: 0.1380 - val_accuracy: 0.9740
Epoch 118/500
274/274 [==============================] - 4s 13ms/step - loss: 0.0923 - accuracy: 0.9867 - val_loss: 0.1377 - val_accuracy: 0.9737
Epoch 119/500
274/274 [==============================] - 4s 13ms/step - loss: 0.0925 - accuracy: 0.9859 - val_loss: 0.1375 - val_accuracy:

274/274 [==============================] - 3s 12ms/step - loss: 0.0904 - accuracy: 0.9889 - val_loss: 0.1362 - val_accuracy: 0.9763
Epoch 170/500
274/274 [==============================] - 3s 11ms/step - loss: 0.0904 - accuracy: 0.9872 - val_loss: 0.1361 - val_accuracy: 0.9760
Epoch 171/500
274/274 [==============================] - 3s 12ms/step - loss: 0.0907 - accuracy: 0.9869 - val_loss: 0.1359 - val_accuracy: 0.9753
Epoch 172/500
274/274 [==============================] - 3s 12ms/step - loss: 0.0907 - accuracy: 0.9867 - val_loss: 0.1366 - val_accuracy: 0.9755
Epoch 173/500
274/274 [==============================] - 3s 12ms/step - loss: 0.0903 - accuracy: 0.9869 - val_loss: 0.1363 - val_accuracy: 0.9761
Epoch 174/500
274/274 [==============================] - 3s 11ms/step - loss: 0.0905 - accuracy: 0.9869 - val_loss: 0.1364 - val_accuracy: 0.9764
Epoch 175/500
274/274 [==============================] - 3s 12ms/step - loss: 0.0909 - accuracy: 0.9859 - val_loss: 0.1361 - val_accuracy:

274/274 [==============================] - 4s 13ms/step - loss: 0.0903 - accuracy: 0.9868 - val_loss: 0.1361 - val_accuracy: 0.9769
Epoch 226/500
274/274 [==============================] - 4s 13ms/step - loss: 0.0906 - accuracy: 0.9869 - val_loss: 0.1360 - val_accuracy: 0.9764
Epoch 227/500
274/274 [==============================] - 4s 14ms/step - loss: 0.0901 - accuracy: 0.9875 - val_loss: 0.1361 - val_accuracy: 0.9767
Epoch 228/500
274/274 [==============================] - 3s 12ms/step - loss: 0.0900 - accuracy: 0.9882 - val_loss: 0.1361 - val_accuracy: 0.9765
Epoch 229/500
274/274 [==============================] - 4s 13ms/step - loss: 0.0902 - accuracy: 0.9867 - val_loss: 0.1362 - val_accuracy: 0.9768
Epoch 230/500
274/274 [==============================] - 4s 13ms/step - loss: 0.0900 - accuracy: 0.9869 - val_loss: 0.1363 - val_accuracy: 0.9765
Epoch 231/500
274/274 [==============================] - 4s 13ms/step - loss: 0.0898 - accuracy: 0.9877 - val_loss: 0.1363 - val_accuracy:

274/274 [==============================] - 3s 12ms/step - loss: 0.0902 - accuracy: 0.9871 - val_loss: 0.1361 - val_accuracy: 0.9767
Epoch 282/500
274/274 [==============================] - 4s 13ms/step - loss: 0.0900 - accuracy: 0.9877 - val_loss: 0.1361 - val_accuracy: 0.9769
Epoch 283/500
274/274 [==============================] - 4s 13ms/step - loss: 0.0901 - accuracy: 0.9873 - val_loss: 0.1361 - val_accuracy: 0.9767
Epoch 1/500
110/110 [==============================] - 2s 12ms/step - loss: 0.2936 - accuracy: 0.9081 - val_loss: 0.3574 - val_accuracy: 0.8957
Epoch 2/500
110/110 [==============================] - 1s 10ms/step - loss: 0.2939 - accuracy: 0.9069 - val_loss: 0.3565 - val_accuracy: 0.8957
Epoch 3/500
110/110 [==============================] - 1s 11ms/step - loss: 0.2955 - accuracy: 0.9064 - val_loss: 0.3556 - val_accuracy: 0.8967
Epoch 4/500
110/110 [==============================] - 1s 10ms/step - loss: 0.2882 - accuracy: 0.9084 - val_loss: 0.3548 - val_accuracy: 0.8973


Epoch 55/500
110/110 [==============================] - 1s 12ms/step - loss: 0.2279 - accuracy: 0.9224 - val_loss: 0.3298 - val_accuracy: 0.8997
Epoch 56/500
110/110 [==============================] - 1s 11ms/step - loss: 0.2248 - accuracy: 0.9217 - val_loss: 0.3295 - val_accuracy: 0.8997
Epoch 57/500
110/110 [==============================] - 1s 11ms/step - loss: 0.2242 - accuracy: 0.9243 - val_loss: 0.3291 - val_accuracy: 0.8997
Epoch 58/500
110/110 [==============================] - 1s 8ms/step - loss: 0.2270 - accuracy: 0.9239 - val_loss: 0.3288 - val_accuracy: 0.8997
Epoch 59/500
110/110 [==============================] - 1s 8ms/step - loss: 0.2213 - accuracy: 0.9229 - val_loss: 0.3284 - val_accuracy: 0.9000
Epoch 60/500
110/110 [==============================] - 1s 10ms/step - loss: 0.2214 - accuracy: 0.9219 - val_loss: 0.3281 - val_accuracy: 0.9003
Epoch 61/500
110/110 [==============================] - 1s 10ms/step - loss: 0.2208 - accuracy: 0.9214 - val_loss: 0.3278 - val_accu

0.8952

In [ ]:
model_eval()

In [ ]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
model = create_simple_net()

model.fit(
    x=train_data_3,
    y=train_labels_3,
    epochs=500,
    verbose=1,
    validation_split=0.3,
    batch_size=64,
    callbacks=[lr_reducer, early_stopper]
)

for i in range(len(model.layers)-1):
    model.layers[i].trainable = False
    
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.00001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')

model.fit(
    x=train_data,
    y=train_labels,
    epochs=500,
    verbose=1,
    validation_split=0.3,
    batch_size=64,
    callbacks=[lr_reducer, early_stopper]
)

model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_fer_resnet_c1.h5')

Epoch 1/500
274/274 [==============================] - 4s 14ms/step - loss: 1.6269 - accuracy: 0.8088 - val_loss: 2.1935 - val_accuracy: 0.6063
Epoch 2/500
274/274 [==============================] - 4s 13ms/step - loss: 1.1957 - accuracy: 0.8609 - val_loss: 0.9842 - val_accuracy: 0.8457
Epoch 3/500
274/274 [==============================] - 3s 11ms/step - loss: 0.9174 - accuracy: 0.8803 - val_loss: 0.8493 - val_accuracy: 0.8808
Epoch 4/500
274/274 [==============================] - 3s 12ms/step - loss: 0.7213 - accuracy: 0.8934 - val_loss: 0.7761 - val_accuracy: 0.8811
Epoch 5/500
274/274 [==============================] - 4s 13ms/step - loss: 0.5359 - accuracy: 0.9089 - val_loss: 0.6201 - val_accuracy: 0.8939
Epoch 6/500
274/274 [==============================] - 4s 13ms/step - loss: 0.4314 - accuracy: 0.9168 - val_loss: 0.5274 - val_accuracy: 0.8948
Epoch 7/500
274/274 [==============================] - 3s 12ms/step - loss: 0.3558 - accuracy: 0.9285 - val_loss: 0.4452 - val_accuracy:

274/274 [==============================] - 3s 12ms/step - loss: 0.1051 - accuracy: 0.9818 - val_loss: 0.2445 - val_accuracy: 0.9269
Epoch 58/500
274/274 [==============================] - 4s 13ms/step - loss: 0.1061 - accuracy: 0.9802 - val_loss: 0.2431 - val_accuracy: 0.9271
Epoch 59/500
274/274 [==============================] - 3s 11ms/step - loss: 0.1050 - accuracy: 0.9810 - val_loss: 0.2437 - val_accuracy: 0.9307
Epoch 60/500
274/274 [==============================] - 3s 13ms/step - loss: 0.1045 - accuracy: 0.9812 - val_loss: 0.2405 - val_accuracy: 0.9293
Epoch 61/500
274/274 [==============================] - 4s 13ms/step - loss: 0.1048 - accuracy: 0.9822 - val_loss: 0.2404 - val_accuracy: 0.9280
Epoch 62/500
274/274 [==============================] - 4s 13ms/step - loss: 0.1028 - accuracy: 0.9833 - val_loss: 0.2438 - val_accuracy: 0.9267
Epoch 63/500
274/274 [==============================] - 4s 13ms/step - loss: 0.1029 - accuracy: 0.9821 - val_loss: 0.2437 - val_accuracy: 0.925

110/110 [==============================] - 2s 13ms/step - loss: 0.2619 - accuracy: 0.9176 - val_loss: 0.3318 - val_accuracy: 0.8987
Epoch 2/500
110/110 [==============================] - 1s 11ms/step - loss: 0.2620 - accuracy: 0.9173 - val_loss: 0.3308 - val_accuracy: 0.8993
Epoch 3/500
110/110 [==============================] - 1s 11ms/step - loss: 0.2641 - accuracy: 0.9136 - val_loss: 0.3300 - val_accuracy: 0.9003
Epoch 4/500
110/110 [==============================] - 1s 11ms/step - loss: 0.2568 - accuracy: 0.9163 - val_loss: 0.3293 - val_accuracy: 0.8997
Epoch 5/500
110/110 [==============================] - 1s 11ms/step - loss: 0.2578 - accuracy: 0.9151 - val_loss: 0.3285 - val_accuracy: 0.9000
Epoch 6/500
110/110 [==============================] - 1s 10ms/step - loss: 0.2534 - accuracy: 0.9183 - val_loss: 0.3278 - val_accuracy: 0.9003
Epoch 7/500
110/110 [==============================] - 1s 9ms/step - loss: 0.2578 - accuracy: 0.9180 - val_loss: 0.3272 - val_accuracy: 0.9017
Epoch

Epoch 58/500
110/110 [==============================] - 1s 9ms/step - loss: 0.1839 - accuracy: 0.9364 - val_loss: 0.3061 - val_accuracy: 0.9067
Epoch 59/500
110/110 [==============================] - 1s 9ms/step - loss: 0.1852 - accuracy: 0.9357 - val_loss: 0.3058 - val_accuracy: 0.9067
Epoch 60/500
110/110 [==============================] - 1s 9ms/step - loss: 0.1803 - accuracy: 0.9367 - val_loss: 0.3056 - val_accuracy: 0.9063
Epoch 61/500
110/110 [==============================] - 1s 9ms/step - loss: 0.1769 - accuracy: 0.9383 - val_loss: 0.3054 - val_accuracy: 0.9067
Epoch 62/500
110/110 [==============================] - 1s 9ms/step - loss: 0.1808 - accuracy: 0.9374 - val_loss: 0.3051 - val_accuracy: 0.9067
Epoch 63/500
110/110 [==============================] - 1s 9ms/step - loss: 0.1828 - accuracy: 0.9383 - val_loss: 0.3048 - val_accuracy: 0.9067
Epoch 64/500
110/110 [==============================] - 1s 8ms/step - loss: 0.1772 - accuracy: 0.9391 - val_loss: 0.3045 - val_accuracy:

0.8974

In [ ]:
model_eval()